In [1]:
import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from GameDayFunctions.fangraphs_projection_2020 import Projection
from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = os.environ['BBPATH']+"Fangraphs/projections/"

In [3]:
player_ranking_zips = Projection(year='2020',model='ZiPS')
player_ranking_steam = Projection(year='2020',model='Steamer')

In [4]:
zips = Draft(player_ranking_zips, number_teams = 12)
steam = Draft(player_ranking_steam, number_teams = 12)

In [5]:
zips.draft_all()

In [6]:
steam.draft_all()

In [7]:
steam.roto_placement

9

In [8]:
zips.roto_standings

0     171.0
9     163.0
7     156.0
1     144.5
10    143.0
4     123.0
6     117.0
11    114.5
3     114.0
8     108.0
2     103.5
5     102.5
dtype: float64

In [21]:
steam.roto_standings

0     180.0
9     162.0
10    153.0
11    140.5
1     130.0
4     125.0
7     122.0
3     122.0
2     116.5
6     115.0
5     114.0
8      80.0
dtype: float64

In [11]:
steam.teams[6]['roster']

{'1B': 'Cody Bellinger',
 'SS': 'Fernando Tatis',
 '2B': 'Ozzie Albies',
 'RP': ['Aroldis Chapman', 'Tyler Duffey'],
 'UTIL': 'Carlos Santana',
 'SP': ['Drew Pomeranz', 'Brandon Woodruff'],
 'P': [['Zack Britton', 'John Brebbia'], 'Colin Poche'],
 'BN': [[[['Yoan López', 'Keynan Middleton'], 'Mark Melancon'], 'Luis Patiño'],
  'Tyler Rogers'],
 '3B': 'Luis Arraez',
 'OF': [['Nick Senzel', 'Willie Calhoun'], 'Franmil Reyes'],
 'C': 'Carson Kelly'}

In [29]:
for tm in steam.teams:
    print(steam.teams[tm]['roster'])

{'OF': [['Christian Yelich', 'Starling Marte'], 'Trey Mancini'], '2B': 'Ketel Marte', '3B': 'Rafael Devers', 'SP': ['Tyler Glasnow', 'Shane Bieber'], '1B': 'Anthony Rizzo', 'RP': ['Ryan Pressly', 'Nick Wittgren'], 'P': [['Chris Paddack', 'José Leclerc'], 'Yonny Chirinos'], 'BN': [[[['Hyun-Jin Ryu', 'Trevor May'], 'Anthony Bass'], 'Daniel Ponce'], 'Blake Treinen'], 'SS': 'Corey Seager', 'UTIL': 'Scott Kingery', 'C': 'Yasmani Grandal'}
{'OF': [['Mookie Betts', 'Marcell Ozuna'], 'Cavan Biggio'], 'SP': ['Walker Buehler', 'Zack Greinke'], 'SS': 'Trea Turner', '3B': 'Yoán Moncada', 'RP': ['Keone Kela', 'Will Harris'], 'P': [['Yusmeiro Petit', 'Rafael Montero'], 'Blake Snell'], 'BN': [[[['Tommy Kahnle', 'Lucas Giolito'], 'Shane Greene'], 'Matt Manning'], 'César Hernández'], '2B': 'Gleyber Torres', 'UTIL': 'Justin Turner', '1B': 'Brandon Lowe', 'C': 'Willson Contreras'}
{'OF': [['Mike Trout', 'Aaron Judge'], 'Jorge Soler'], 'SP': ['Jack Flaherty', 'Mike Clevinger'], 'RP': ['Roberto Osuna', 'Wi

In [712]:
path_yahoo = '/Users/marcoviero/data/baseball/Fangraphs/projections/2020/Yahoo/'
daya = pd.read_table(path_yahoo+'Yahoo_Roto_Ranking_2020.txt')

In [823]:
yahoo_df = pd.DataFrame(columns=['Rank','PLAYER','ELIG. POS.'])
for i in np.arange(len(daya)):
    posn = daya.iloc[i].values[0].split(' - ')
    prts = posn[0].split(' ')
    d = {'Rank':float(prts[0]), 'PLAYER':str(prts[1]+' '+prts[2]), 'ELIG. POS.':posn[1].replace(',','/')}
    yahoo_df = yahoo_df.append(d, ignore_index = True)

In [821]:
yahoo_df_sorted = yahoo_df.iloc[yahoo_df.Rank.argsort()]

In [822]:
yahoo_df_sorted.to_csv(path_yahoo+'Yahoo_Roto_Ranking_2020.csv',index=False)